In [1]:
# import internal modules
import file_path_management as fpath
import public_library as plib
import extract_info
import parameters as params
import download_and_process_pdf as dp

2023-09-10 17:43:39 Didis-MacBook-Pro.local metapub.config[22474] WARNING NCBI_API_KEY was not set.


In [2]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [3]:
# extract text from given .pdf file
def pdf2text(pdf_path, text_path, page_start, page_end):
    try:
        # creating a pdf reader object
        reader = PyPDF2.PdfReader(pdf_path)
        
        # printing number of pages in pdf file
        page_max = len(reader.pages)
        
        # getting a specific page from the pdf file
        text = ""
        
        for i in range(page_start, page_end + 1):
            # print(i)
            page = reader.pages[i]
            text = text + "".join(page.extract_text().splitlines())

        with open(text_path, "w") as f:
            f.write(text)
        f.close()

        return True
    except:
        return False
# --------------------start of test code--------------------
# ind = 1
# pdf_file_name = str(index) + ".pdf"
# pdf_folder = fpath.litera_pdf_folder
# pdf_path = os.path.join(pdf_folder, pdf_file_name)

# text_folder_path = fpath.litera_text_folder
# text_file_name = pdf_file_name.split(".pdf")[0] + ".txt"
# text_path = os.path.join(text_folder_path, text_file_name)

# start_page = 0
# end_page = 1
# if not pdf2text(pdf_path, text_path, start_page, end_page):
#     print("Error: pdf2text() failed")
# ---------------------end of test code---------------------

In [4]:
# download pdf
def download_pdf(pdf_url, pdf_source, pdf_folder):
    for website in params.website_pdfs:
        func = None
        if website in pdf_source:
            # Get the function name by replacing "." with "_" and use globals() to call it
            func_name = "download_from_" + website.replace(".", "_")
            # print(func_name)
            func = globals().get(func_name)
            # print(func)
            break
    if func != None:
        # print(func)
        func(pdf_url, pdf_folder)
    else:
        print("The given url is not from a supported website: ", pdf_url)
        raise Exception("Function does not exist for website:", pdf_url)

# rename downloaded pdf
def rename_pdf(ind, pdf_folder, time_to_wait=60):
    newname = str(ind) + ".pdf"
    time_counter = 0
    filename = max([f for f in os.listdir(pdf_folder)], key=lambda xa :   os.path.getctime(os.path.join(pdf_folder,xa)))
    while '.part' in filename:
        time.sleep(1)
        time_counter += 1
        if time_counter > time_to_wait:
            raise Exception('Waited too long for file to download')
    filename = max([f for f in os.listdir(pdf_folder)], key=lambda xa :   os.path.getctime(os.path.join(pdf_folder,xa)))
    os.rename(os.path.join(pdf_folder, filename), os.path.join(pdf_folder, newname))

# download and rename pdf
def download_and_rename_pdf(pdf_url, ind, pdf_folder):  
    if pdf_url != pdf_url:
        raise Exception("pdf_url is np.nan")
    else:
        pdf_source = pdf_url.split("://")[1].split("/")[0]
    download_pdf(pdf_url, pdf_source, pdf_folder)
    rename_pdf(ind, pdf_folder, time_to_wait=60)
    print(f"Downloaded {ind}.pdf")
# --------------------start of test code--------------------
# pdf_url = "https://journals.physiology.org/doi/pdf/10.1152/jn.2001.85.1.219"
# ind = 10
# pdf_folder = "/home/hou/myProjects/litera_pdfs"
# download_and_rename_pdf(pdf_url, ind, pdf_folder)
# ---------------------end of test code---------------------
# https://journals.physiology.org/doi/10.1152/jn.2001.85.1.219

In [5]:
# full_text_url = 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5856726/'
# doi = '10.1007/s12264-018-0210-1'
# pmid = '29430586'
# pmcid = 'PMC5856726'
# if full_text_url == full_text_url and full_text_url.split("://")[1].split("/")[0] == 'www.ncbi.nlm.nih.gov':
#     if doi == doi:
#         url = "https://doi.org/" + doi
#         url, status_code = plib.get_final_redirected_url(url)
#         info = extract_info.extract_info_from_webpage(url, params.websites)
#         full_text_url = url
#         full_text_source = url.split("://")[1].split("/")[0]
#         pdf_url = info["pdf_link"]
#         pdf_source = pdf_url.split("://")[1].split("/")[0]
#     else:
#         pdf_url = np.nan
#         pdf_source = np.nan
# else:  
#     print("PDF_URL not found for: ", doi, pmid, pmcid, full_text_url)
#     pdf_url = np.nan
#     pdf_source = np.nan
# print(doi)
# print(pmid)
# print(pmcid)
# print(full_text_url)
# print(full_text_source)
# print(pdf_url)
# print(pdf_source)

In [6]:
# # www.microbiologyresearch.org
# "://www.microbiologyresearch.org/"
# "https://www.microbiologyresearch.org/content/journal/jgv/10.1099/vir.0.79883-0"
# # europepmc.org
# "://europepmc.org/"
# "https://europepmc.org/article/med/8784824"
# # jamanetwork.com
# "://jamanetwork.com/"
# "https://jamanetwork.com/journals/jamaneurology/article-abstract/565945"
# # papers.ssrn.com
# "://papers.ssrn.com/"
# "https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3689615"
# # royalsocietypublishing.org
# "://royalsocietypublishing.org/"
# "https://royalsocietypublishing.org/doi/10.1098/rspb.1953.0054"
# # www.ingentaconnect.com
# "://www.ingentaconnect.com/"
# "https://www.ingentaconnect.com/content/aalas/cm/2000/00000050/00000002/art00006"
# # journals.lww.com
# "://journals.lww.com/"
# "https://journals.lww.com/neuroreport/abstract/1994/10000/further_evidence_for_two_types_of_corticopulvinar.6.aspx"

In [7]:
# # "//button[@class='dropdown-trigger btn btn--light btn--cta_roundedColored']"
# ['www.ahajournals.org', 'psychiatryonline.org']
# download pdf to specified folder given pdf_url and ind
def download_pdf_by_buttun(url, ind):  
    # try:
        # set up the webdriver
        os.environ['WDM_LOG'] = '0'
        options = Options()
        options.add_argument('--headless')
        driver = webdriver.Firefox(options=options)

        # load the webpage
        # error_label = 0
        # while(error_label == 0):
            # try:
        driver.get(url)
        element = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class,'dropdown-trigger btn')]")))
        element.click()
        time.sleep(10)
        element = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "//ul[contains(@class,'base-download-options')]/li[1]/a"))).get_attribute("href")
        
        time.sleep(10)
        driver.quit()
                # error_label = 1
            # except:
            #     print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            #     time.sleep(5*60)
            #     error_label = 0
    #     return True
    # except:
        return False
# --------------------start of test code--------------------
# # 'www.ahajournals.org'
# pdf_url = "https://www.ahajournals.org/doi/reader/10.1161/01.STR.0000087786.38997.9E"
# # 'neuro.psychiatryonline.org'
# # 'ajp.psychiatryonline.org'
# # pdf_url = "https://neuro.psychiatryonline.org/doi/reader/10.1176/jnp.16.2.127"
# # pdf_url = "https://ajp.psychiatryonline.org/doi/reader/10.1176/appi.ajp.158.9.1411"
# ind = 3
# pdf_folder = fpath.pdf_folder
# if pdf_url.split("://")[1].split("/")[0] == 'www.ahajournals.org':
#     url = "https://www.ahajournals.org/doi/epdf/" + pdf_url.split("reader/")[1]
# elif "psychiatryonline.org" in pdf_url.split("://")[1].split("/")[0]:
#     url = pdf_url

# if download_pdf_by_buttun(url, ind):
#     print(f'Successfully downloaded PDF:', ind)
# else:
#     print(f'Failed downloading PDF:', ind)
# ---------------------end of test code---------------------

In [8]:
# driver.get(url)
['iovs.arvojournals.org', 'www.imrpress.com', 'www.hifo.uzh.ch', 'ujms.net', 'www.annualreviews.org', 'thejns.org']
def download_pdf_by_driver(url, ind, pdf_folder): 
    # set up the webdriver
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')

    options.set_preference("browser.download.folderList", 2)
    options.set_preference("browser.download.manager.showWhenStarting", False)
    options.set_preference("browser.download.dir", pdf_folder)
    options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf")
    options.set_preference("pdfjs.disabled", True)

    driver = webdriver.Firefox(options=options)
    driver.set_page_load_timeout(5)

    try:
        driver.get(url)
        return True
    except TimeoutException:
        # print("Page load timed out but that's okay!")
        return True
    except:
        return False
    finally:
        driver.quit()
# --------------------start of test code--------------------
# # iovs.arvojournals.org
pdf_url = "https://iovs.arvojournals.org//arvo/content_public/journal/iovs/934840/i1552-5783-57-1-1.pdf"
# # www.imrpress.com
# pdf_url = "https://www.imrpress.com/journal/JIN/20/1/10.31083/j.jin.2021.01.334/pdf"
# # www.hifo.uzh.ch
# pdf_url = "https://www.hifo.uzh.ch/dam/jcr:00000000-2999-c151-ffff-ffffd9509b89/paper_CorticalArea8.pdf"
# # ujms.net
# pdf_url = "https://ujms.net/index.php/ujms/article/download/6812/12603"
# # www.annualreviews.org
# pdf_url = "https://www.annualreviews.org/doi/pdf/10.1146/annurev.ne.11.030188.001345"
# # thejns.org
# pdf_url = "https://thejns.org/downloadpdf/journals/j-neurosurg/86/1/article-p77.xml"
ind = 4
pdf_folder = fpath.pdf_folder
if download_pdf_by_driver(pdf_url, ind, pdf_folder):
    print(f'Successfully downloaded PDF:', ind)
else:
    print(f'Failed downloading PDF:', ind)
# ---------------------end of test code---------------------

Successfully downloaded PDF: 4
